In [ ]:
import os
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import nengo
from scipy import stats
from nengo.utils.matplotlib import rasterplot
from nengo.utils.stdlib import Timer

import phd

# Some plotting niceties
plt.rc('figure', figsize=(10, 6))
sns.set_style('white')
sns.set_style('ticks')

def img(array, zscore=False):
    if zscore:
        array = stats.zscore(array, axis=0)
    plt.pcolormesh(array.T)
    plt.ylim(top=array.shape[1])
    plt.xlim(right=array.shape[0])
    plt.yticks(())
    sns.despine(left=True)
    plt.tight_layout()

def img_compare(array1, array2, zscore=(False, False)):
    plt.figure(figsize=(12, 6))
    plt.subplot(1, 2, 1)
    img(array1, zscore[0])
    plt.subplot(1, 2, 2)
    img(array2, zscore[1])
    plt.tight_layout()

timit_path = "~/phd_data/timit"
timit = phd.timit.TIMIT(timit_path)
try:
    timit.untar(os.path.expanduser("~/Dropbox/LDC93S1.tgz"))
except Exception as e:
    pass

In [ ]:
%%javascript
if($(IPython.toolbar.selector.concat(' > #kill-run-first')).length == 0){
  IPython.toolbar.add_buttons_group([
    {
      'label'   : 'kill and run-first',
      'icon'    : 'fa fa-angle-double-down',
      'callback': function(){
        IPython.notebook.kernel.restart();
        $(IPython.events).one('kernel_ready.Kernel', function(){
          var idx = IPython.notebook.get_selected_index();
          IPython.notebook.select(0);
          IPython.notebook.execute_cell();
          IPython.notebook.select(idx);
        });
      }
    }
  ], 'kill-run-first');
}

## On a single sample

In [ ]:
timit = phd.timit.TIMIT(timit_path)
timit.filefilt.spkr_id = "CAG0"
samples = timit.word_samples(['wash'])
model = phd.sermo.AuditoryFeatures()
model.audio = samples['wash'][0]

In [ ]:
# Get MFCCs
model.mfcc.dt = 0.01
x = model.mfcc()
print x.shape
img_compare(x, x, zscore=(False, True))

In [ ]:
# Get NCCs
net = model.build()
with net:
    ihc_p = nengo.Probe(net.periphery.ihc, synapse=None)
    an_in_p = nengo.Probe(net.periphery.an.input, synapse=None)
    an_p = nengo.Probe(net.periphery.an.add_neuron_output(), synapse=None)
    c_p = nengo.Probe(net.output, synapse=0.01)
sim = nengo.Simulator(net, dt=0.001)
sim.run(model.t_audio)

img_compare(sim.data[c_p], sim.data[c_p], zscore=(False, True))

In [ ]:
# A peek into the neural implementation
plt.figure(figsize=(8, 14))
plt.subplot(3, 1, 1)
phd.plots.cochleogram(sim.data[ihc_p], sim.trange(), model.freqs)
plt.subplot(3, 1, 2)
phd.plots.cochleogram(sim.data[an_in_p], sim.trange(), model.freqs)
plt.subplot(3, 1, 3)
rasterplot(sim.trange(), sim.data[an_p])
plt.ylim(0, net.periphery.an.n_neurons * net.periphery.an.n_ensembles)
sns.despine()
plt.tight_layout()

## With derivatives

In [ ]:
timit = phd.timit.TIMIT(timit_path)
timit.filefilt.spkr_id = "CAG0"
samples = timit.word_samples(['wash'])
model = phd.sermo.AuditoryFeatures()
model.audio = samples['wash'][0]
model.add_derivative()  # First derivative
model.add_derivative()  # Second derivative

In [ ]:
# Get MFCCs
model.mfcc.dt = 0.01
x = model.mfcc()
print x.shape
img_compare(x, x, zscore=(False, True))

In [ ]:
# Get NCCs
net = model.build()
with net:
    ihc_p = nengo.Probe(net.periphery.ihc, synapse=None)
    an_in_p = nengo.Probe(net.periphery.an.input, synapse=None)
    an_p = nengo.Probe(net.periphery.an.add_neuron_output(), synapse=None)
    c_p = nengo.Probe(net.output, synapse=0.01)
sim = nengo.Simulator(net, dt=0.001)
sim.run(model.t_audio)

img_compare(sim.data[c_p], sim.data[c_p], zscore=(False, True))

## With Adaptive LIF neurons in AN

In [ ]:
timit = phd.timit.TIMIT(timit_path)
timit.filefilt.spkr_id = "CAG0"
samples = timit.word_samples(['wash'])
model = phd.sermo.AuditoryFeatures()
model.audio = samples['wash'][0]
model.periphery.adaptive_neurons = True

In [ ]:
# Get MFCCs
model.mfcc.dt = 0.01
x = model.mfcc()
print x.shape
img_compare(x, x, zscore=(False, True))

In [ ]:
# Get NCCs
net = model.build()
with net:
    ihc_p = nengo.Probe(net.periphery.ihc, synapse=None)
    an_in_p = nengo.Probe(net.periphery.an.input, synapse=None)
    an_p = nengo.Probe(net.periphery.an.add_neuron_output(), synapse=None)
    c_p = nengo.Probe(net.output, synapse=0.01)
sim = nengo.Simulator(net, dt=0.001)
sim.run(model.t_audio)

img_compare(sim.data[c_p], sim.data[c_p], zscore=(False, True))

## Phoneme classification with SVM

Since the samples have to be the same length,
we classify vowel and consonant phonemes separately.
Also, we lengthen all samples to be the
length of the longest sample
with simple linear interpolation.

In [ ]:
# No derivatives
model = phd.sermo.AuditoryFeatures()
expt = phd.experiments.AuditoryFeaturesExperiment(model, phones=phd.timit.TIMIT.phones)
expt.seed = 9
expt.timit.filefilt.region = 8
expt.cache_key

In [ ]:
# With 2 derivatives
model = phd.sermo.AuditoryFeatures()
model.add_derivative(klass="FeedforwardDeriv")
model.add_derivative(klass="FeedforwardDeriv")
expt = phd.experiments.AuditoryFeaturesExperiment(model, seed=0, phones=phd.timit.TIMIT.consonants)
expt.timit.filefilt.region = 1
result = expt.run()
print "Accuracy with MFCC:", result.mfcc_acc
print "Accuracy with NCC:", result.ncc_acc

In [ ]:
# With 1 derivative and z-score
model = phd.sermo.AuditoryFeatures()
model.add_derivative(klass="FeedforwardDeriv")
expt = phd.experiments.AuditoryFeaturesExperiment(model, seed=0, phones=phd.timit.TIMIT.consonants)
expt.zscore = True
expt.timit.filefilt.region = 1
result = expt.run()
print "Accuracy with MFCC:", result.mfcc_acc
print "Accuracy with NCC:", result.ncc_acc

## Results

In [ ]:
# Phoneme experiment: vowels vs consonants vs all
from phd import analysis
analysis = reload(analysis)

In [ ]:
klass = phd.experiments.AuditoryFeaturesResult
res = analysis.load_results(klass, ['phones'])

In [ ]:
res

In [ ]:
df['mfcc_time'][0]

In [ ]:
import pandas as pd

resds = []
for key in res:
    resd = [r.__dict__ for r in res[key]]
    for d in resd:
        d['phones'] = key[0]
    resds.append(pd.DataFrame(resd))

df = pd.concat(resds, ignore_index=True)

for key in klass.to_numeric:
    df[key] = df[key].apply(float)

df

In [ ]:
# g = sns.PairGrid(df,
#                  x_vars=["mfcc_train_acc", "ncc_train_acc", "mfcc_test_acc", "ncc_test_acc"],
#                  y_vars=["phones"])
# g.map(sns.violinplot)
plt.subplot(1, 2, 1)
sns.violinplot(x="phones", y="mfcc_train_acc", data=df)
plt.ylim(0, 1)
plt.subplot(1, 2, 2)
sns.violinplot(x="phones", y="ncc_train_acc", data=df)
plt.ylim(0, 1)
plt.figure()
plt.subplot(1, 2, 1)
sns.violinplot(x="phones", y="mfcc_test_acc", data=df)
plt.ylim(0, 1)
plt.subplot(1, 2, 2)
sns.violinplot(x="phones", y="ncc_test_acc", data=df)
plt.ylim(0, 1)